## mindspore.ops.max(input, axis=None, keepdims=False, *, initial=None, where=None) -〉 tuple (Tensor)
返回给定轴上的最大值及其索引。
- 输入：
    * input: mindspore的tensor。
    * axis: int。
    * keepdims: bool。
    * initial: scalar
    * where:Tensor[bool]

- 返回：2个mindspore的tensor组成的tuple, 最大值及最大值的索引。

1、参数比较：
| mindspore   | torch       | jax         |
| :----:      | :----:      | :----:      |
| input       | input       | a           |
| axis        | dim         | axis        |
| keepdims    | keepdim     | keepdims    |
| initial     | out         | initial     |
| where       |             | where       |
|             |             | out         |


* torch和jax都提供了out出参方式
* torch没有initial和where参数，分别表示返回值的初始值及是否要计算最大值。

2、返回值比较

In [5]:
import numpy as np
import mindspore as ms
import torch
import jax.numpy as jnp

input = np.array([[9, 3, 4, 5],
                  [5, 2, 7, 4],
                  [8, 1, 3, 6]])

y1 = ms.ops.max(ms.tensor(input), 1)
y2 = torch.max(torch.tensor(input), 1)
y3 = jnp.max(input, 1)
print ('mindspore output:\n',y1)
print('\n')
print ('torch output:\n',y2)
print('\n')
print ('jax output:\n',y3)

mindspore output:
 (Tensor(shape=[3], dtype=Int64, value= [9, 7, 8]), Tensor(shape=[3], dtype=Int64, value= [0, 2, 0]))


torch output:
 torch.return_types.max(
values=tensor([9, 7, 8]),
indices=tensor([0, 2, 0]))


jax output:
 [9 7 8]


* ms返回的信息不简洁，关键信息value在最后面，可读性差。  
* jax仅返回最大值，不返回最大值索引。

使用where，来确定计算范围时：

In [8]:
where=np.array([[0, 0, 0, 0],
                [0, 1, 0, 1],
                [1, 1, 1, 0]], dtype=bool)

y1 = ms.ops.max(ms.tensor(input), 1, keepdims=True, initial=0, where=ms.tensor(where))
y3 = jnp.max(input, 1, keepdims=True, initial=0, where=where)
print ('mindspore output:\n',y1)
print('\n')
print ('jax output:\n',y3)

mindspore output:
 (Tensor(shape=[3, 1], dtype=Int64, value=
[[0],
 [4],
 [8]]), Tensor(shape=[3, 1], dtype=Int64, value=
[[0],
 [3],
 [0]]))


jax output:
 [[0]
 [4]
 [8]]


3、报错信息比较

In [9]:
y1 = ms.ops.max(input, 1)

TypeError: Failed calling ArgMaxWithValue with "ArgMaxWithValue(axis=int, keep_dims=bool)(input=<class 'numpy.ndarray'>)".
The valid calling should be: 
"ArgMaxWithValue(axis=<int>, keep_dims=<bool>)(input=<Tensor>)".

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore/ccsrc/pipeline/pynative/pynative_utils.cc:1294 PrintTypeCastError


In [10]:
y2 = torch.max(input, 1)

TypeError: max() received an invalid combination of arguments - got (numpy.ndarray, int), but expected one of:
 * (Tensor input, *, Tensor out)
 * (Tensor input, Tensor other, *, Tensor out)
 * (Tensor input, int dim, bool keepdim, *, tuple of Tensors out)
 * (Tensor input, name dim, bool keepdim, *, tuple of Tensors out)


In [11]:
input = [2, 4, 3, 1]
y3 = jnp.max(input, 1)

TypeError: max requires ndarray or scalar arguments, got <class 'list'> at position 0.

当输入类型不正确时，报错信息torch简洁明确。建议ms优化。